In [ ]:
import torch
model_name = "googlenet"

correlations = torch.load(
    "/mnt/disk1/user/Tue.CM210908/ICLR2025/correlation/correlations_integrated_gradients_googlenet.pth", weights_only=False
)
print(correlations)

import matplotlib.pyplot as plt
from matplotlib import cm

markers = ['o', 's', 'D', '^', 'v']
plt.figure(figsize=(7, 4.8) if model_name == "googlenet" else (10, 4.8))
if model_name == "resnet50":
    plot_layers = ["layer4.2", "layer4.1", "layer4.0", "layer3.5", "layer3.4", 
                   "layer3.3", "layer3.2", "layer3.1", "layer3.0", "layer2.3", 
                   "layer2.2", "layer2.1", "layer2.0", "layer1.2", "layer1.1",
                   "layer1.0"]
elif model_name == "googlenet":
    plot_layers = ["inception5b", "inception5a", "inception4e", "inception4d",
                   "inception4c", "inception4b", "inception4a", "inception3b",
                   "inception3a"]
else:
    raise ValueError(f"Unsupported model name {model_name}")

setups = correlations[0].keys()
colors = cm.get_cmap('viridis_r', len(setups)+1)
available = ['default'] + plt.style.available
with plt.style.context(available[17]):
    x = range(len(plot_layers))
    
    for i, setup in enumerate(setups):
        y = [d[setup] for d in correlations]
        plt.plot(x, y, label=f"Masking $S_r = {setup}$", color=colors(i+1), 
                 marker=markers[i % len(markers)], markersize=7)
    
    if model_name == "resnet50":
        abbreviated_layers = [layer.replace('layer', '') for layer in plot_layers]
    else:
        abbreviated_layers = [layer.replace('inception', '') for layer in plot_layers]
    plt.xticks(x, abbreviated_layers, fontsize=20 if model_name == "resnet50" else 20)
    plt.yticks(fontsize=18)
    plt.xlabel('Layers', fontsize=22)
    plt.ylabel('Correlation', fontsize=22)
    plt.title(f'Correlation - {model_name}', fontsize=28)
    plt.ylim(-1.05, 0.05)
    plt.legend(fontsize=16)
    
    plt.tight_layout()
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from matplotlib import cm

markers = ['o', 's', 'D', '^', 'v']
models = ["resnet50", "googlenet"]
subtitles = ["ResNet50", "GoogLeNet"]
available = ['default'] + plt.style.available

with plt.style.context(available[17]):
    # Adjust the width ratios for the subplots
    fig, axes = plt.subplots(1, 2, figsize=(17, 7), gridspec_kw={'width_ratios': [1.4, 1]})
    for idx, model_name in enumerate(models):
        correlations = torch.load(f"./correlation/correlations_integrated_gradients_{model_name}.pth", weights_only=False)
        
        if model_name == "resnet50":
            plot_layers = ["layer4.2", "", "layer4.0", "", "layer3.4", 
                        "", "layer3.2", "", "layer3.0", "", 
                        "layer2.2", "", "layer2.0", "", "layer1.1",
                        ""]
        elif model_name == "googlenet":
            plot_layers = ["inception5b", "", "inception4e", "",
                        "inception4c", "", "inception4a", "",
                        "inception3a"]
        else:
            raise ValueError(f"Model {model_name} not supported")
        
        setups = correlations[0].keys()
        colors = cm.get_cmap('viridis_r', len(setups)+1)
        ax = axes[idx]
        if model_name == "resnet50":
            ax.set_facecolor('#f5edf3') 
        if model_name == "googlenet":
            ax.set_facecolor('#e7f1e6')
        x = range(len(plot_layers))
        
        for i, setup in enumerate(setups):
            y = [d[setup] for d in correlations]
            ax.plot(x, y, label=f"$\\tau = {setup}$", color=colors(i+1), 
                    marker=markers[i % len(markers)], markersize=12, linestyle='-' if idx==0 else "-")
        
        if model_name == "resnet50":
            abbreviated_layers = [layer.replace('layer', '') for layer in plot_layers]
        else:
            abbreviated_layers = [layer.replace('inception', '') for layer in plot_layers]
        
        ax.set_xticks(x)
        fontsize_xticks = 26
        ax.set_xticklabels(abbreviated_layers, fontsize=fontsize_xticks)
        ax.tick_params(axis='y', labelsize=26)
        ax.set_xlabel('Layers', fontsize=26)
        if idx == 0:
            ax.set_ylabel('Correlation', fontsize=30)
        ax.set_title(f'{subtitles[idx]}', fontsize=30)
        ax.set_ylim(-1.05, -0.1)
        ax.grid(True, axis='y', linestyle='--', alpha=0.6)

    # Combine legends from both subplots
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, fontsize=28, loc='lower center', ncol=len(setups), bbox_to_anchor=(0.5, -0.05)) # type: ignore
    plt.tight_layout(rect=[0, 0.05, 1, 0.95]) # type: ignore
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from matplotlib import cm

# Define model-specific configurations
model_names = ["resnet50", "googlenet"]
markers = ['o', 's', 'D', '^', 'v']
plot_layers_dict = {
    "googlenet": ["inception5b", "", "inception4e", "", "inception4c", "", "inception4a", "", "inception3a"],
    "resnet50": ["layer4.2", "", "layer4.0", "", "layer3.4", "", "layer3.2", "", "layer3.0"]
}
subtitles_dict = {
    "googlenet": "GoogLeNet",
    "resnet50": "ResNet50"
}
# method_names = ["integrated_gradients", "guided_backprop", "lrp", "saliency", "knockoff", "smoothgrad", "gradient_shap"]
method_names = ["integrated_gradients"]
subtitles = ["Integrated Gradients", "Guided Backpropagation", "LRP", "Saliency", "Knockoff", "SmoothGrad", "Gradient Shap"]

# Set plotting style
available_styles = ['default'] + plt.style.available

# Begin plotting
with plt.style.context(available_styles[17]): 
    fig, axes = plt.subplots(1, 2, figsize=(14, 4.5), sharey=True)
    lines = [] 
    for i, model_name in enumerate(model_names):
        ax = axes[i]
        subtitle = subtitles_dict[model_name]
        plot_layers = plot_layers_dict[model_name]
        x = range(len(plot_layers))
        colors = cm.get_cmap('plasma', len(method_names) + 1)

        for idx, method_name in enumerate(method_names):
            # Load correlations for the current method and model
            correlations = torch.load(f"./correlation/correlations_{method_name}_{model_name}.pth", weights_only=False)
            setups = correlations[0].keys()
            y_values = []

            for idx_layer in x:
                d = correlations[idx_layer]
                y_list = [d[setup] for setup in setups]
                y_avg = sum(y_list) / len(y_list)
                y_values.append(y_avg)

            line, = ax.plot(x, y_values, label=subtitles[idx],
                            marker=markers[idx % len(markers)], markersize=10,
                            color=colors(idx))

            # Add the line to the list for the shared legend
            if i == 0:  # Only need to add the lines from one subplot
                lines.append(line)

        # Format X-axis labels
        abbreviated_layers = [layer.replace('layer', '').replace('inception', '') for layer in plot_layers]
        if model_name == "resnet50":
            axes[i].set_facecolor('#f5edf3') 
        if model_name == "googlenet":
            axes[i].set_facecolor('#e7f1e6')
        
        ax.set_xticks(range(len(plot_layers)))
        ax.set_xticklabels(abbreviated_layers, fontsize=20)
        ax.tick_params(axis='y', labelsize=20)
        if i == 0:
            ax.set_ylabel('Average Correlation', fontsize=24)
        ax.set_title(f'{subtitle}', fontsize=28)
        ax.set_ylim(-1.05, 0.1)
        ax.grid(True, axis='y', linestyle='--', alpha=0.6)

    plt.subplots_adjust(wspace=10) 
    fig.legend(lines, subtitles, fontsize=20, loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=4)
    fig.text(0.54, -0.05, 'Layers', ha='center', fontsize=24)

    # Tight layout for better spacing
    plt.tight_layout()
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from matplotlib import cm

markers = ['o', 's', 'D', '^', 'v']
models = ["resnet50", "googlenet"]
subtitles = ["ResNet50", "GoogLeNet"]
available = ['default'] + plt.style.available

with plt.style.context(available[17]):
    # Adjust the width ratios for the subplots
    fig, axes = plt.subplots(1, 2, figsize=(17, 7), gridspec_kw={'width_ratios': [1.4, 1]})
    for idx, model_name in enumerate(models):
    
        if model_name == "resnet50":
            plot_layers = ["layer4.2", "", "layer4.0", "", "layer3.4", 
                        "", "layer3.2", "", "layer3.0", "", 
                        "layer2.2", "", "layer2.0", "", "layer1.1",
                        ""]
        elif model_name == "googlenet":
            plot_layers = ["inception5b", "", "inception4e", "",
                        "inception4c", "", "inception4a", "",
                        "inception3a"]
        else:
            raise ValueError(f"Model {model_name} not supported")
        
        correlations = torch.load("./correlation/" + f"correlations_integrated_gradients_{model_name}_compare_sum_and_mean.pth", weights_only=False)
        setups = correlations[0].keys()
        colors = cm.get_cmap('viridis_r', len(setups)+1)
        ax = axes[idx]
        if model_name == "resnet50":
            ax.set_facecolor('#f5edf3') 
        if model_name == "googlenet":
            ax.set_facecolor('#e7f1e6')
        x = range(len(plot_layers))
        
        for i, setup in enumerate(setups):
            y = [d[setup] for d in correlations]
            ax.plot(x, y, label="Average" if setup=="True" else "Original", color=colors(i+1), 
                    marker=markers[i % len(markers)], markersize=10, linewidth=3)
        
        if model_name == "resnet50":
            abbreviated_layers = [layer.replace('layer', '') for layer in plot_layers]
        else:
            abbreviated_layers = [layer.replace('inception', '') for layer in plot_layers]
        
        ax.set_xticks(x)
        fontsize_xticks = 26
        ax.set_xticklabels(abbreviated_layers, fontsize=fontsize_xticks)
        ax.tick_params(axis='y', labelsize=26)
        ax.set_xlabel('Layers', fontsize=26)
        if idx == 0:
            ax.set_ylabel('Correlation', fontsize=30)
        ax.set_title(f'{subtitles[idx]}', fontsize=30)
        ax.set_ylim(-1.05, -0.1)
        ax.grid(True, axis='y', linestyle='--', alpha=0.6)

    # Combine legends from both subplots
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, fontsize=28, loc='lower center', ncol=len(setups), bbox_to_anchor=(0.5, -0.05))

    plt.tight_layout(rect=[0, 0.05, 1, 0.95]) # type: ignore
    plt.show()